# Workshop PR03 (optional): Other ways of setting up workspace

Recall from workshop PR01, we have set up workspace on our own computer. Now there are other ways of setting up workspace for faster and better ML performance. 

## Option 2: Setting up Jupter Notebook with Docker 

### 1.) Conventional Docker image for Jupyter-Notebook 
First install Docker. Then pull the jupyter image from Docker Hub if not present on local machine and then starts the server on port 8888. This is done by the following simple command, 

> $ docker run -d -P --name notebook jupyter/all-spark-notebook

### 2.) Building your own custom Docker image
If you want to specify a seperate customised Docker image, then build a docker image from the following Dockerfile. To build the image go to the directory location of the Dockerfile and type the following command to build the image, 

> $ Docker build -t img_name .   

In [ ]:
#########The Dockerfile ########

# get the latests repo from ubuntu
FROM "ubuntu:latest"
RUN apt-get update && yes | apt-get upgrade > /dev/null 2>&1


# installations of python packages 
RUN apt-get -qq install -y git python3-pip python3-dev protobuf-compiler python-pil python-lxml libsm6 libxrender1 libfontconfig1 > /dev/null 2>&1


# python modules
RUN pip3 install tensorflow > /dev/null 2>&1
RUN pip3 install opencv-contrib-python > /dev/null 2>&1
RUN pip3 install jupyter > /dev/null 2>&1
RUN pip3 install matplotlib > /dev/null 2>&1
RUN pip3 install pillow > /dev/null 2>&1


# PYTHONPATH augment the default search path for module files.
RUN export PYTHONPATH=$PYTHONPATH:pwd:pwd/slim
RUN jupyter notebook --generate-config --allow-root
RUN echo "c.NotebookApp.password = u'sha1:6a3f528eec40:6e896b6e4828f525a6e20e5411cd1c8075d68619'" >> /root/.jupyter/jupyter_notebook_config.py


EXPOSE 8888
CMD ["jupyter", "notebook", "--allow-root", "--notebook-dir=/tensorflow/models/research/object_detection", "--port=8888", "--no-browser", "--ip=0.0.0.0"]

## Option 3: GCP instance

#### Step 1:
Please refer the following link for signing up to Google Cloud Console, 

https://accounts.google.com/signin/v2/identifier?service=cloudconsole&passive=1209600&osid=1&continue=https%3A%2F%2Fconsole.cloud.google.com%2Fcompute&followup=https%3A%2F%2Fconsole.cloud.google.com%2Fcompute&flowName=GlifWebSignIn&flowEntry=ServiceLogin

After entering you credit card details (You will not be charged untill you exceed your $400 credit limit, plus when you exceed the credit limit you get a notification so dont have to worry)  and complete the sign up process. 

#### Step 2:

Once you complete the sign up in step one go to **compute engine >> VM instances** 
and click on "Create new instance". Once you're in the VM details page, enter the following details for the VM instance Details

| Option                   |      chose                                           | 
| ------------------------ |-----------------------------------------------------:| 
| **Boot Disk**            | Ubuntu 16.04 LTS                                     |
| **Zone**                 | Australia (any where)                                | 
| **CPU's**                | Number of CPUs as you wish, but the upper limit is 8 | 
| **Identify & API Access**| Tick allow default access                            |
| **Firewall**             | Click Allow HTTP & HTTPS traffic                     |
| **Deletion Rule**        | Keep it unticked                                     |

Once you enter the details Click "Create", DONE!!!! Now your cloud instance is created 

**IMPORTANT: Now you should see instance-1 (whatever the instance name you chose) with a green tick indicating the instance is running. Click on the 3 dots at the end, followed by STOP option to stop the instance. If you do not not stop it the instance will keep on running and you will be charged on hourly basis (instance with high performance will cost even more on hourly basis)**

**Step 3:**

Now make the ecternal IP of the VM instance static for convenient ssh access. ( You will not be charged for static IP as long as you frequantly use the VM instance). Guide your self to **VPC Network >> External IP addresses** and change the tpye from **Euphemerical** to **Static**

**Step 4:**

Now giude youre self to **VPC Network >> Firewall Rule** and click "Create New Rule" then add the following specifications,

| Option                   |      chose                              | 
| ------------------------ |----------------------------------------:| 
| **Network**              | Default                                 |
| **priority**             | 1000                                    | 
| **Direction**            | Ingress                                 | 
| **Action on Match**      | Allow                                   |
| **Target**               | All instances on the network            |
| **Source Filter**        | IP Ranges                               |
| **Source IP Ranges**     | 0.0.0.0/0                               |
| **Second Source Filter** | None                                    |
| **Protocols and Ports**  | Specified protocols and ports > tcp:5000|

#### Step 5:

Now Proceed to you VM instances page ( Compute engine >> VM Instances ) and start the VM instance. Give it a few seconds, then proceed to click **SSH** button to start the shell session. Now a pop up shell will appear and we will do some more configuration in the cloud instance. 

i) Download Anaconda:
> $ wget http://repo.continuum.io/archive/Anaconda3-4.0.0-Linux-x86_64.sh

ii) Install Anaconda: (Click yes to all once you enter the below command)
> $ bash Anaconda3-4.0.0-Linux-x86_64.sh
    
iii) Source it:
> $ source ~/.bashrc
   
iv) Update Anaconda for latest version:
> $ conda update anaconda

v) install the main ML framework which has many important dependencies:
> $ pip install tensorflow

vi) generate jupyter configuration file 
> $ jupyter notebook --generate-config
    
vii) Edit the configuration file

> $ nano ~/.jupyter/jupyter_notebook_config.py
    
and copy paste following lines 
    
    c = get_config()
    c.NotebookApp.ip = '*'
    c.NotebookApp.open_browser = False
    c.NotebookApp.port = <Port Number> # since we allow port 5000 in the firewall rule, put 5000
 
**DONE** Now type the follwoing to start up the jupyter server 

> $ jupyter-notebook --no-browser --port=5000
    
Now it should start a jupyter session on your chrome browser 

#### Step 5: (Optional )

This step will show you a conveneint way of manageing all your VM instances from your local machine through the gcloud SDK. This is also very useful than the cloud shell session, as it is much much faster. It also allowes you to have multiple SSH sessions for managing files, monitoring CPU load and etc 




Follow the link and install the specific gcloud SDK version on your computer 

https://cloud.google.com/sdk/docs/quickstart-macos (This is for MacOS)

i) Following installation initialize gcloud as follows, this will giude you to sign into your GCP account, 

> $ gcloud init    

(Windows/Linux/Mac: Install git from here https://git-scm.com/downloads)

ii) Generate a public SSH key in the VM instance ssh keys page, If you do have a SSH key already skip to the following step, 

> $ ssh-keygen -t rsa -b 4096 -C "your_email@example.com"

iii) Now copy  the SSH key to your clipboard (Windows: Install git from here https://git-scm.com/downloads)

> $ pbcopy < ~/.ssh/id_rsa.pub

    
iv) Go to GCP >> VM instances >> edit >> edit SSH keys, and paste the SSH key. replace last bit of the SSH key with your e-mail prefix. The click save. 

v) Now go to your local machine and type, this will list you all the available VM instances and their status 

> $ gcloud compute instances list 
    
vi) Now start the VM instance you want by typing the following command, 

> $ gcloud compute instances EXTERNAL_IP_of_VM
    
vii) After theinstance is running, type the following command to SSH, 

> $ ssh username@EXTERNAL_IP_of_VM

viii) Then inside the shell type, 

> $ jupyter-notebook --no-browser --port=5000
    
ix) Go to your web browser and type the following in the URL bar,

> $ EXTERNAL_IP_of_VM:5000
 
